In [1]:
import notion
from bs4 import BeautifulSoup
from markdownify import markdownify as md
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from notion.client import *
from notion.block import *
from notion.client import NotionClient


In [2]:
## RUN THIS CELL

def AccessNotion():
    #Security
    global client
    try:
        client = NotionClient(token_v2 = token)
    except:
        pass
    while ('client' not in globals()) or type(client) != notion.client.NotionClient:
        token = input("Please paste your Notion token: ")
        try:
            client = NotionClient(token_v2 = token)
        except:
            print("Something went wrong. Is your token correct?")
    print("ACCESS GRANTED.")

def GetCollectionView():
    global cv
    # Get Colletions
    try:
        cv = client.get_collection_view(link_cv)
    except:
        pass
    while ('cv' not in globals()) or ((type(cv) != notion.collection.BoardView) and (type(cv) !=notion.collection.TableView)):
        link_cv = input("Please paste link to collection view: ")
        try:
            cv = client.get_collection_view(link_cv)
        except:
            print("Something went wrong. Is your link correct?")
    print("DATABASE FOUND.")

def CreatePage():
    print("Creating Page...")
    global session
    session = cv.collection.add_row()
    


    # Go to recently created session page
    global page
    page = client.get_block(session.id)
    page.title = soup.find('h1').text

    

    # Clean Page
    for block in page.children:
        block.remove()

    #In-class Notes
    CreateInclassNotes()
    

    # Readings
    CreateReadings()
    
    # Study Guide
    CreateStudyGuide()

    # Pre-class
    CreatePreclass()
    
    # Session Properties
    DefineProperties()
    
    print("You page is ready! Access it here: " + page.get_browseable_url())
            
def CreateInclassNotes():
    global page
    print("Formatting In-class Notes...")
    notes = page.children.add_new(SubheaderBlock,
                                  title="In-class Notes",
                                  color="yellow")
    divider0 = page.children.add_new(DividerBlock)
    page.children.add_new(TextBlock, title="")
    print("Done \n")
    
    
def CreateReadings():    
    global count
    print("Formatting Readings...")
    readings = page.children.add_new(SubheaderBlock, title="Readings", color="orange")
    divider1 = page.children.add_new(DividerBlock)

    readings_titles = soup.find_all('h4')[1:]
    descriptions = soup.find("div", {"class": "description underlined-links"})
    count = 0
    global links
    links = []
    for title in readings_titles:
        print("   - " + title.text)
        CreateReading(title)
    print("Done \n")
    

def CreateReading(title):
        global count
        global links
        quote = title.find_next_sibling("blockquote")
        if title.find_next_sibling().name == "p" and title.find_next_sibling().findChild().name == "a":
            count+=1
            try:
                link = title.find_next_sibling("p").findChild()
                links.append(link.get("href"))
                cur_reading = page.children.add_new(ToggleBlock,
                                  title="[{0}]({1})".format(title.text,
                                                            link.text))
            except:
                cur_reading = page.children.add_new(ToggleBlock, title = title.text)
        else:
            cur_reading = page.children.add_new(ToggleBlock, title = title.text)

        # Content of toggle    
        cur_reading.children.add_new(TextBlock, title="**Objective:**", color = "grey")
        cur_reading.children.add_new(TextBlock, title=html_to_markdown(quote.text))
        cur_reading.children.add_new(TextBlock, title="**Notes:**")
        cur_reading.children.add_new(TextBlock, title="")
        cur_reading.children.add_new(TextBlock, title="**Summary:**")
        cur_reading.children.add_new(TextBlock, title="")
        
def CreateStudyGuide(color = "blue"):
    print("Formatting Study Guide...")
    study_guide = page.children.add_new(SubheaderBlock,
                                        title="Study Guide",
                                        color=color)
    divider2 = page.children.add_new(DividerBlock)

    prestudy = soup.find_all('h3')[3]
    study_text = prestudy.find_previous_siblings(['p', 'pre'])

    page.children.add_new(TextBlock, title=html_to_markdown(study_text[:-count]))
    print("Done \n")
    
def CreatePreclass(color = "green"):
    print("Formatting Pre-Class...")
    Preclass = page.children.add_new(SubheaderBlock, title="Pre-class", color=color)
    divider2 = page.children.add_new(DividerBlock)

    prepreclass = soup.find_all('h3')[3]
    preclass_text = prepreclass.find_next_siblings()
    preclass_mark = html_to_markdown(preclass_text)

    page.children.add_new(TextBlock, title=preclass_mark)
    print("Done \n")
    
def DefineProperties():
    global session
    print("Defining Class Properties...")
    session.name = soup.find('h1').text[1:]
    session.course = soup.find('h1').text.split()[0]
    session.class_link = soup.find('h1').findChild('a').get('href')
    session.materials = links
    session.reviewed = False
    hcs = ["#" + i.text.split("-")[-1] for i in soup.find_all('a', "hash-link")]
    
    try:
        session.hcs_los = hcs
    except:
        session.hcs_los = ", ".join(hcs)
    print("Done \n")
    
def html_to_markdown(html):
    markdown = ''
    for i in html:
        markdown+=md(str(i).replace("<code>", "`").replace("</code>", "`"), strip = ['pre']) 
    return markdown

def login():
    driver.get("https://forum.minerva.kgi.edu/")
    
    main_window_handle = None
    while not main_window_handle:
        main_window_handle = driver.current_window_handle
    time.sleep(5)
    login_button = driver.find_element_by_xpath("//button[1]")
    login_button.click()

    signin_window_handle = None
    while not signin_window_handle:
        for handle in driver.window_handles:
            if handle != main_window_handle:
                signin_window_handle = handle
                break

    driver.switch_to.window(signin_window_handle)
    
    mail_address = input("E-mail: ")
    password = input("Password: ")
    
    try:
        
        driver.find_element_by_id("identifierId").send_keys(mail_address)
        driver.find_element_by_id("identifierNext").click()
        driver.implicitly_wait(2)


        driver.find_element_by_name("password").send_keys(password)
        driver.find_element_by_id("passwordNext").click()
    except:
        "Wrong credentials"

    time.sleep(10)
    driver.switch_to.window(main_window_handle)
    

In [3]:
# Optionally, to speed up the process, define the token_v2 and the collection view below

token = "bb12053b78080cbbddb53cf3e9350c9a3648355915fc643c11aa548acc89c593ce6a39cdaeb94dcc390d4ca404144a4dd74a61472cf86e5bca66b35a4d8e9490bc2e074433ce0c251d3920600571" # Puth the token_v2 here as a string
link_cv = "https://www.notion.so/felipesworkshop/9a6986af5d8043c790a653b8327dfa70?v=dce2ce4555ca4c4a84cfd000ce41c8b7" # Put the collection view link here as a string

In [5]:
## Run this cell to start the program

AccessNotion()

GetCollectionView()

### AUTOMATIC LOGIN, (currently not working) ### 

# try:
#     del test_login
# except:
#     pass
    
# while ("test_login" not in locals()) or (test_login == None):
#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     wait = WebDriverWait(driver, 10)
#     login()

    
#     soup = BeautifulSoup(driver.page_source, "html.parser")
#     test_login = soup.find('a', 'navigation-link ')
#     if test_login == None:
#         print("Something went wrong with the login")

###    ########  ####

# Opening Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://forum.minerva.kgi.edu/")
print("Go to the newly opened Chrome page and login to Forum. After that, come back to the notebook and follow the code outputs.")

# Wait untl login
try:
    element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "navigation-link "))
    )
except:
    print("You took too long to login")
    driver.quit()


make_another = "Yes"
url = input("URL to class (type 'stop' to exit): ")
while make_another == 'Yes':
    try:
        driver.get(url)
    except:
        print("Couldn't find page with this url.")
        continue

    load = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "assignment-description"))
    )
    
    # Scraping page
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Create New Notion Page
    class_link = soup.find('h1').findChild('a').get('href')
    if class_link in [row.class_link for row in cv.collection.get_rows()]:
        make_duplicate = input("Class already exists. Do you want to make a copy? (Yes/No)")
        if make_duplicate == "Yes":
            CreatePage()
    else:
        CreatePage()
        
    while make_another != ("Yes" or "No"):
        make_another = input("Another class? (Yes/No): ")
    

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\rober\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


ACCESS GRANTED.
DATABASE FOUND.
 
Go to the newly opened Chrome page and login to Forum. After that, come back to the notebook and follow the code outputs.
URL to class (type 'stop' to exit): https://forum.minerva.kgi.edu/app/courses/1641/sections/7017/classes/50800


KeyboardInterrupt: 

In [6]:
url = input("URL to class (type 'stop' to exit): ")
while make_another == 'Yes':
    try:
        driver.get(url)
    except:
        print("Couldn't find page with this url.")
        continue

    load = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CLASS_NAME, "assignment-description")))
    
    # Scraping page
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Create New Notion Page
    class_link = soup.find('h1').findChild('a').get('href')
    if class_link in [row.class_link for row in cv.collection.get_rows()]:
        make_duplicate = input("Class already exists. Do you want to make a copy? (Yes/No)")
        if make_duplicate == "Yes":
            CreatePage()
    else:
        CreatePage()
        
    make_another = input("Another class? (Yes/No): ")

URL to class (type 'stop' to exit): https://forum.minerva.kgi.edu/app/courses/1641/sections/7017/classes/50800
Creating Page...
Formatting In-class Notes...
Done 

Formatting Readings...
   - CS110 course guide, academic year 2020-2021


C:\Users\rober\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


   - Part I, section 1.1 of Cormen, T. H., Leiserson, C. E., Rivest, R. L., & Stein, C. (2009). Introduction to algorithms(3rd ed.). Cambridge, MA: MIT Press.
   - Bhargava, A. (2016) Grooking Algorithms. Chapter 3, pages 37-41
Done 

Formatting Study Guide...
Done 

Formatting Pre-Class...
Done 

Defining Class Properties...
Done 

You page is ready! Access it here: https://www.notion.so/790e275496684bc09f3a4c27f6b2d623
Class already exists. Do you want to make a copy? (Yes/No)No
Class already exists. Do you want to make a copy? (Yes/No)Yes
Creating Page...


KeyboardInterrupt: 